<a href="https://colab.research.google.com/github/yifang828/ASR-and-summarization/blob/main/Trump_whisper_ASR_and_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade pytube
!pip install yt-dlp
!pip install openai-whisper
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s et

In [7]:
# 下載川普總統關稅宣布音檔
# !wget https://drive.google.com/file/d/1huHDCsQjVcmedqubEK28I36ti99lO9IJ/view?usp=sharing -O trump.mp4
!pip install -U gdown

import gdown

file_id = "1huHDCsQjVcmedqubEK28I36ti99lO9IJ"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "trump.mp4", quiet=False)



Downloading...
From: https://drive.google.com/uc?id=1huHDCsQjVcmedqubEK28I36ti99lO9IJ
To: /content/trump.mp4
100%|██████████| 69.7M/69.7M [00:01<00:00, 45.6MB/s]


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [1]:
# 使用whisper將trump.mp4轉成逐字稿(ASR),儲存至trump.txt,會耗時7-9分鐘
import whisper


model = whisper.load_model("medium")
result = model.transcribe("./trump.mp4",)
# print(result["text"])

f = open("./trump.txt", "w", encoding='utf-8')
f.write(result["text"])
f.close()

In [ ]:
# gemini做summarization:為避免內容太長，分段給LLM做摘要
import textwrap

def read_text(src_path):
  with open(src_path, "r", encoding='utf-8') as f:
    text = f.read()
    return text

def make_chunks(text, chunk_size):

  return textwrap.wrap(text, chunk_size)

In [ ]:
# 按照給的chunck_size將內容分成多個小段落

SRC_PATH = "./trump.txt"
CHUNCK_SIZE = 512

text = read_text(SRC_PATH)
chunks = make_chunks(text, CHUNCK_SIZE)



## 透過 gemini multi-stage summarization
分成多個段落各自摘要，再將每個小摘要合併做一個總結摘要

In [ ]:
import google.generativeai as genai
import time
from time import sleep

def setup_gemini(api_key, model_type, top_k, tempareture, top_p, max_token):
  genai.configure(api_key=api_key)
  model = genai.GenerativeModel(model_type, generation_config={"temperature":tempareture,
                                                               "top_p":top_p,
                                                               "max_output_tokens": max_token})
  return model


def summarize(model, user_prompt):
    start = time.time()
    response = model.generate_content(user_prompt)
    print(f"elapsed time:{time.time()-start}")
    sleep(3)
    return response.text


In [ ]:
# ===========================TODO===========================
# 請將LLM_API_KEY解註解並改成你自己的 gemini LLM_API_KEY

# LLM_API_KEY = "YOUR GMEINI API KEY"
MODEL = "gemini-2.0-flash-lite"
PARAGRAPH = 2000
SUMMARY_PATH = "./trump_summary.txt"

total_paragraph = []
model = setup_gemini(api_key=LLM_API_KEY, model_type=MODEL, top_k=1, tempareture=0, top_p=1, max_token=PARAGRAPH)

for i, chunk in enumerate(chunks):
  user_prompt=f"""
  以下是一篇關於川普關稅演講
  請用中文整理演講的重點，總長度控制在 200 字內並包含：
  1. 主題與背景說明
  2. 核心議題或結論
  3. 若有，請簡述涉及的重要數據或趨勢
  4. 語氣專業、清晰，適合商務報告使用, 演講原文:{chunk}"""

  response = summarize(model, user_prompt)
  print(f"第{i+1}段摘要: {response}")
  total_paragraph.append(response)

# 再將每個小摘要合併做一個總結摘要
user_prompt=f"""
  以下是一篇關於川普關稅演講
  請用中文整理演講的重點，總長度控制在 2000 字內並包含：
  1. 主題與背景說明
  2. 核心議題或結論
  3. 若有，請簡述涉及的重要數據或趨勢
  4. 語氣專業、清晰，適合商務報告使用, 演講原文:{total_paragraph}"""

all_summarize = summarize(model, user_prompt)
with open(SUMMARY_PATH, "w", encoding='utf-8') as f:
  f.write(all_summarize)

elapsed time:1.7493083477020264
第1段摘要: 川普總統於 2025 年 4 月 2 日發表了關於貿易政策的演講，主題為「解放日」，旨在宣告美國產業的復興。演講核心議題是透過關稅政策，重塑美國經濟，使國家再次富裕。演講中，川普總統強調了對美國經濟前景的樂觀預期，並將此日視為美國命運的轉捩點。然而，演講中並未提及具體的數據或趨勢分析，而是著重於激勵人心的口號和願景。

elapsed time:1.7678437232971191
第2段摘要: 本次演講聚焦於美國貿易逆差問題，背景為美國長期遭受不公平貿易待遇，導致國內產業衰退。

核心議題為重建美國製造業，保護美國勞工權益。演講強調，外國政府透過不公平手段，竊取美國就業機會，破壞美國經濟。

演講中提及美國鋼鐵工人、汽車工人、農民和技術工人等群體，暗示這些行業遭受嚴重衝擊。演講者表達了對美國夢的懷念，並承諾將採取措施扭轉此趨勢，重建美國經濟實力。

elapsed time:1.5667812824249268
第3段摘要: 川普總統的演講聚焦於貿易關稅政策，核心議題是終止美國長期以來遭受的貿易剝削。演講強調，美國及其納稅人已遭受超過 50 年的貿易不公。

演講中，川普總統宣布將簽署一項歷史性的行政命令，對全球各國實施互惠關稅。這意味著，若他國對美國產品徵收關稅，美國也將對其產品徵收同等關稅。

elapsed time:1.7425239086151123
第4段摘要: 本次演講主題為「經濟獨立宣言」，旨在闡述美國經濟政策轉向。演講背景為美國長期以來在國際貿易中遭受損失，導致國內經濟發展受限。

核心議題是透過關稅手段，保護美國本土產業，實現經濟繁榮。演講強調將運用數兆美元資金，用於減稅及償還國債，並承諾快速實現經濟目標。

演講未提及具體數據，但暗示將透過政策調整，扭轉過去的經濟劣勢，使美國經濟重回增長軌道。

elapsed time:1.6168205738067627
第5段摘要: 本次演講聚焦於川普政府的貿易政策，旨在重振美國經濟。演講核心議題為透過關稅手段，促進國內就業與製造業回歸，並打破外國貿易壁壘。演講強調，透過增加國內生產，將強化市場競爭，降低消費者價格，最終實現「美國黃金時代」。演講預期國內產業將迎來強勁復甦，並積極推動美國經濟的再次崛起。

elapsed ti

分成多個段落摘要，每個新段落的摘要都會考量先前的摘要

In [ ]:
# ===========================TODO===========================
# 請將LLM_API_KEY解註解並改成你自己的 gemini LLM_API_KEY

# LLM_API_KEY = "YOUR GMEINI API KEY"
MODEL = "gemini-2.0-flash-lite"
PARAGRAPH = 2000
REFINEMENT_SUMMARY_PATH = "./trump_refinement_summary.txt"

total_paragraph = ""
model = setup_gemini(api_key=LLM_API_KEY, model_type=MODEL, top_k=1, tempareture=0, top_p=1, max_token=PARAGRAPH)

for i, chunk in enumerate(chunks):
  if i == 0:
    user_prompt=f"""
    以下是一篇關於川普關稅演講
    請用中文整理演講的重點，總長度控制在 200 字內並包含：
    1. 主題與背景說明
    2. 核心議題或結論
    3. 若有，請簡述涉及的重要數據或趨勢
    4. 語氣專業、清晰，適合商務報告使用, 演講原文:{chunk}"""

    response = summarize(model, user_prompt)
    print(f"第{i+1}段摘要: {response}")
    total_paragraph = response

  else:
    user_prompt=f"""
      請結合先前的摘要:{total_paragraph},
      將以下關於川普關稅演講內容
      用中文整重點，總長度控制在 2000 字內並包含：
      1. 主題與背景說明
      2. 核心議題或結論
      3. 若有，請簡述涉及的重要數據或趨勢
      4. 語氣專業、清晰，適合商務報告使用, 演講原文:{chunk}"""

    response = summarize(model, user_prompt)
    print(f"第{i+1}段摘要: {response}")
    total_paragraph = response

with open(REFINEMENT_SUMMARY_PATH, "w", encoding='utf-8') as f:
  f.write(total_paragraph)

elapsed time:1.742262601852417
第1段摘要: 本次演講聚焦於美國產業復興，宣布2025年4月2日為「解放日」，象徵美國產業的重生與國家命運的重塑。演講核心議題為透過貿易政策，使美國再次富強。演講者強調將透過相關措施，扭轉過去數十年來的貿易逆差，並帶領美國走向經濟繁榮。然而，演講中並未明確提及具體數據或趨勢，僅表達對未來經濟發展的樂觀預期。

elapsed time:4.763000011444092
第2段摘要: ## 川普關稅演講重點摘要：美國產業復興與貿易政策

**1. 主題與背景說明**

本次演講聚焦於美國產業的復興，並將2025年4月2日定為「解放日」，象徵美國產業的重生與國家命運的重塑。演講背景指出，美國長期以來遭受來自國內外，包括盟友和敵人的「掠奪、劫掠和蹂躪」。演講者特別強調，美國的鋼鐵工人、汽車工人、農民和技術工人遭受了嚴重的痛苦，他們目睹了外國領導人竊取美國的就業機會，外國欺詐者洗劫工廠，以及外國掠奪者摧毀了曾經美好的美國夢。

**2. 核心議題或結論**

演講的核心議題是透過貿易政策，使美國再次富強。演講者承諾將採取相關措施，扭轉過去數十年來的貿易逆差，帶領美國走向經濟繁榮。演講的核心結論是，透過保護主義的貿易政策，重建美國的產業基礎，恢復美國的經濟實力，並重塑美國的國家形象。

**3. 重要數據或趨勢 (演講中未明確提及)**

儘管演講中並未明確提及具體數據或趨勢，但其背後的邏輯暗示了以下幾點：

*   **貿易逆差的嚴重性：** 演講暗示美國長期以來面臨嚴重的貿易逆差，這被視為導致美國產業衰退和就業機會流失的主要原因。
*   **產業空心化：** 演講中提及工廠被洗劫，暗示美國產業空心化，製造業外移，導致國內就業機會減少。
*   **經濟衰退的擔憂：** 演講中對美國經濟現狀的負面描述，反映了對經濟衰退的擔憂，以及對現有貿易政策的質疑。
*   **保護主義的必要性：** 演講強調了保護主義貿易政策的重要性，例如關稅，以保護國內產業免受外國競爭的影響。

**4. 總結**

本次演講的核心目標是透過貿易政策，重建美國的產業基礎，並重塑美國的經濟實力。演講者將貿易逆差、產業空心化和就業機會流失歸咎於現有的貿易政策，並承諾透過保護主義措施，扭轉這一趨勢。雖然演講中並未提供具體的數據支持，但其背後